## Imports

TODO: 
- Unir los datos de ambas competiciones
- Intentar política one cycle
- Más data augmentation
- Sin RandomCrop
- Con el data augmentation de brillo y saturación

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import *
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from skimage import io, transform
from PIL import Image
import PIL
import cv2

import pandas as pd
import numpy as np

from pathlib import Path
import os 
import random

path1 = Path("../input/cassava-leaf-disease-classification")
path2 = Path("../input/cassava-leaf-disease-merged")

## Data preprocessing

In [ ]:
data = pd.read_csv(path/"train.csv")
data.head()

In [ ]:
count = [len(data[data.label == i]) for i in np.sort(data.label.unique())]
state = ["Bacterias", "Veta marrón", "Moteado verde", "Mosaico", "Sano"]
for i,j in enumerate(count):
    print(str(state[i]) + ": " + str(j) + ", lo que supone un " + str(round(j*100/len(data), 2)) + "% del total.")

In [ ]:
data_oh = pd.concat([data.image_id, pd.get_dummies(data["label"])], axis=1)
data_oh.columns = ["image_id", "Bacterias", "Veta marrón", "Moteado verde", "Mosaico", "Sano"]
data_oh.head()

In [ ]:
data_oh.to_csv("train_oh.csv", index_label=False)

## Dataset

In [ ]:
def split_from_csv(path_csv, pct_split):
    total_data = pd.read_csv(path_csv)
    # Borrar si hay solo una columna
    total_data = total_data.drop('source', 1)
    valid_data = total_data.sample(frac=pct_split)
    idxs = list(set(list(range(len(total_data)))) - set(valid_data.index))
    train_data = total_data.iloc[idxs]

    return train_data.reset_index(drop=True), valid_data.reset_index(drop=True)

In [ ]:
t, v = split_from_csv("./train_oh.csv", 0.2)
len(t), len(v)

In [ ]:
t, v = split_from_csv(path2/"merged.csv", 0.2)
len(t), len(v)

Creating a dataset class, which preload all images from a pandas dataframe.

In [ ]:
class Dataset_from_csv(Dataset):
    """
    Dataset creado teniendo las imágenes en una carpeta y las etiquetas en un 
    csv en el que se mencionan los nombres de las imágenes de entrada a las que 
    corresponde cada etiqueta.
    """
    def __init__(self, path, dataframe, dir_images, transforms=None):
        """
        Args:
            path (Path Object): Ruta del directorio principal
            csv_file (string): Nombre del csv.
            dir_images (string): Ruta a la carpeta con las imágenes.
            transforms (callable, optional): Lista de posibles transformaciones
                hecha con Compose. Debe acabar con ToTensor.
        """
        self.path = path
        #dataframe =  dataframe[:(int(len(dataframe)/10))] # Temporal
        self.labels = dataframe
        self.path_images = dir_images
        self.transforms = transforms

        """#Cargamos las imágenes para facilitar la velocidad del entrenamiento
        self.loaded_images = {}
        count = 0
        print("Inicio de la carga de archivos.")
        for name in dataframe.image_id:
            if (count%500)==1: print(f"Progreso: {round(count*100/len(dataframe), 2)}%")
            route = path/(str(dir_images) + "/" + str(name))
            image = cv2.imread(os.path.join(route))
            image = None
            count += 1"""

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = self.labels.iloc[idx, 0]
        route = self.path/(str(self.path_images) + "/" + str(img_name))
        image = PIL.Image.open(os.path.join(route))
        im_label = self.labels.iloc[idx].values[1:].astype(float).reshape(-1,1) # reshape(-1,2) si son varias etiquetas
        sample = {"image": image, "label": im_label}

        if self.transforms:
            sample["image"] = self.transforms(sample["image"])

        return sample

Calculating mean and std

In [ ]:
folder = "train_images"
to_tensor = transforms.Compose([
        transforms.Resize(size2),
        transforms.ToTensor(),
        transforms.Normalize((0.4307, 0.4967, 0.3135), (0.1960, 0.2011, 0.1767))
    ])

train_dataset = Dataset_from_csv(path, t, folder,to_tensor)
dataloader = DataLoader(train_dataset, batch_size=int(len(t)*0.035), shuffle=False, num_workers=0)

batch = next(iter(dataloader))
print(batch["image"].size())
b = batch["image"].view(batch["image"].size(0), batch["image"].size(1), -1)
mean = (b.mean(2).sum(0))/batch["image"].size()[0]
std = (b.std(2).sum(0))/batch["image"].size()[0]
print(mean, std)

mean: [0.0222, 0.0253, 0.0109] 

std: [1.0582, 1.0520, 1.0556]

## Neural Networks

### Pretrained EfficientNet with BCE

In [ ]:
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

def NewEfficientNet(c_out=1):
    model = EfficientNet.from_pretrained('efficientnet-b5')
    model._fc = nn.Linear(in_features=2048, out_features=c_out, bias=True)
    return model

model = NewEfficientNet(5)
model = model.cuda()

In [ ]:
model

In [ ]:
def accuracy(y_pred, y):
    n_true = torch.sum(y_pred.data.max(1, keepdim=True)[1].squeeze(-1) == y).item()
    n_samples = y.size()[0]
    return n_true, n_samples

def multilabel_accuracy(y_pred, y, threshold):
    n_true, n_samples = 0, len(y_pred)
    for pred,target in zip(y_pred, y):
        if len(pred[pred>threshold])>1:
            pred_label = (pred==(pred[pred>threshold].max())).nonzero()
        else:
            pred_label = ((pred>threshold)==True).nonzero()
        label = (target==True).nonzero()
        if str(pred_label)==str(label): n_true += 1
    return n_true, n_samples

class Learner():
    def __init__(self, model, train, valid, optim, loss_fn):
        self.model = model
        self.train = train
        self.valid = valid
        self.optim = optim
        self.loss_fn = loss_fn
        self.lr = 0
        self.losses_train = []
        self.losses_valid = []

    def fit(self, epochs, lr, max_lr, threshold):
        #scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optim, max_lr=max_lr, steps_per_epoch=len(self.train), epochs=epochs)
        if lr != self.lr:
            self.lr == lr
            for param_group in self.optim.param_groups:
                param_group['lr'] = lr
        
        for epoch in range(epochs):
            sum_loss = 0
            acc = 0
            correct = 0
            n_samples = 0
            
            acc_loss_val = 0

            # Training
            self.model = self.model.train()
            for count, sample in enumerate(self.train):
                X_batch = sample["image"].to("cuda:0")
                y_batch = sample["label"].long().to("cuda:0")
                y_pred = self.model(X_batch)                
                loss = self.loss_fn(y_pred, y_batch.squeeze(2).squeeze(1))
                correct += (y_batch.squeeze(2).squeeze(1) == y_pred.argmax(axis=1)).sum().item()
                n_samples += len(y_batch.squeeze(2))
 
                if count % 200 == 0:
                    print(f"Train epoch {epoch+1}: [{count}/{len(self.train)}\tLoss: {round(loss.item(), 3)}\tAcc: {round((correct/n_samples)*100, 2)}%]")
                sum_loss += loss.item()
                acc_train = correct/n_samples

                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
                #scheduler.step()

            # Validation
            self.model = self.model.eval()
            correct = 0
            n_samples = 0
            with torch.no_grad():
                for sample_val in self.valid:
                    X_batch = sample_val["image"].to("cuda:0")
                    y_batch = sample_val["label"].long().to("cuda:0")
                    y_pred = self.model(X_batch)
                    loss = self.loss_fn(y_pred, y_batch.squeeze(2).squeeze(1))
                    correct += (y_batch.squeeze(2).squeeze(1) == y_pred.argmax(axis=1)).sum().item()
                    n_samples += len(y_batch.squeeze(2).argmax(axis=1))
                    acc_loss_val += loss.item()
            acc_val = correct/n_samples
            print(f"Train epoch {epoch+1}: Train -> [Loss: {round(sum_loss/len(self.train), 3)}\tAcc: {round(acc_train*100, 2)}%], Valid -> [Loss: {round(acc_loss_val/len(self.valid), 3)}\tAcc: {round(acc_val*100, 2)}%]\n")
            self.losses_train.append(sum_loss/len(self.train))
            self.losses_valid.append(acc_loss_val/len(self.valid))

    def plot(self):
        plt.plot(learner.losses_train)
        plt.plot(learner.losses_valid)
        plt.ylabel('Error')
        plt.xlabel('Epochs')
        plt.legend(["Entrenamiento", "Validación"])
        plt.show()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
size1 = [600, 800]
size2 = [300, 400]
size3 = [150, 200]

size_efficientnet = [528, 528]

current_size = size_efficientnet

tsfm = transforms.Compose([
        transforms.CenterCrop(current_size),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
        transforms.Normalize((0.4307, 0.4967, 0.3135), (0.1960, 0.2011, 0.1767))
    ])

tsfm2 = transforms.Compose([
        transforms.CenterCrop(current_size),
        transforms.ToTensor(),
        transforms.Normalize((0.4307, 0.4967, 0.3135), (0.1960, 0.2011, 0.1767))
    ])

folder = "train"
train_dataset = Dataset_from_csv(path2, t, folder,tsfm)
valid_dataset = Dataset_from_csv(path2, v, folder,tsfm)

In [ ]:
b_size1 = [4,8]
b_size2 = [6,12]
b_size3 = [104,208]

b_size = b_size1

train_dataloader = DataLoader(train_dataset, batch_size=b_size[0], shuffle=True, num_workers=0)
valid_dataloader = DataLoader(valid_dataset, batch_size=b_size[1], shuffle=True, num_workers=0)

In [ ]:
model.load_state_dict(torch.load("./model_3_"))
learner.model = model

In [ ]:
learner = Learner(model, train_dataloader, valid_dataloader, optimizer, loss_fn)

In [ ]:
learner.train=train_dataloader
learner.valid=valid_dataloader
learner.fit(2, 0.00003, 0.1, 0.5)

In [ ]:
learner.fit(2, 0.000005, 0.1, 0.5)

In [ ]:
learner.fit(1, 0.000001, 0.1, 0.5)

In [ ]:
learner.plot()

92.6%